<a href="https://colab.research.google.com/github/gahyunson/food_management/blob/main/food_cnnmodeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data load

In [ ]:
!pip3 install pickle5

     |████████████████████████████████| 256 kB 7.1 MB/s 


In [ ]:
import pickle5 as pickle
import os
import numpy as np
import pandas as pd


In [ ]:
#현재 주소
path = os.getcwd() + '/drive/MyDrive/final_project/pickle'
path

'/content/drive/MyDrive/final_project/pickle'

In [ ]:
# 각 넘버에 따른 음식이름 
food_dict = {'01011001' : '쌀밥', '12011008' : '배추김치', '04016001': '부대찌개', '01015013' : '카레', '01015010' : '제육덮밥'}

In [ ]:
# 푸드 이미지 배열화 시킨 객체 load
with open(path+'/x_pickle/food.pickle', 'rb') as f:
    new_data = pickle.load(f)

In [ ]:
new_data.shape

(1500, 128, 128, 3)

In [ ]:
# labeling data load
with open(path+'/y_pickle/food_label1.pickle', 'rb') as f:
    temp_list = pickle.load(f)

In [ ]:
np.array(temp_list).shape

(1500,)

In [ ]:
# labeling data -> get dummies
df_temp = pd.get_dummies(temp_list)

In [ ]:
 # dummy labeling -> numpy array labeling
 label = np.array(df_temp)

In [ ]:
label.shape

(1500, 5)

In [ ]:
label

array([[0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       ...,
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0]], dtype=uint8)

In [ ]:
food_index = {0:'01011001', 1:'01015010', 2:'01015013', 3:'04016001', 4:'12011008'}

# 모델링

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Conv2D, Dense, MaxPooling2D, Flatten, Dropout, Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import np_utils

from keras.preprocessing import image
#파일 이미지 깨진 부분 오류 메세지 처리 
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True 
from PIL import Image

In [ ]:
# 모델 튜닝 고민해야 할 점

#1. Conv2D를 128, 64 로 ? input_shape가 128, 128 인데 64는 너무 축소...
#2. layer은 2개로 충분한가 ?
#3. Dropout은 얼마로 정할까.

In [ ]:
model=Sequential()
model.add(Conv2D(128 ,kernel_size=(3,3), input_shape=(128,128,3),activation='relu', padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
model.add(Conv2D(32  ,kernel_size=(3,3),activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(label.shape[1], activation='softmax'))

In [ ]:
#환경 설정
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
history=model.fit(new_data,label, epochs=100, batch_size=40)

Epoch 1/100
38/38 [==============================] - 151s 4s/step - loss: 1.4825 - accuracy: 0.3327
Epoch 2/100
38/38 [==============================] - 148s 4s/step - loss: 1.1389 - accuracy: 0.4933
Epoch 3/100
38/38 [==============================] - 151s 4s/step - loss: 1.0669 - accuracy: 0.5133
Epoch 4/100
38/38 [==============================] - 148s 4s/step - loss: 1.0311 - accuracy: 0.5460
Epoch 5/100
38/38 [==============================] - 147s 4s/step - loss: 0.9818 - accuracy: 0.5647
Epoch 6/100
38/38 [==============================] - 147s 4s/step - loss: 0.9418 - accuracy: 0.5980
Epoch 7/100
38/38 [==============================] - 150s 4s/step - loss: 0.8720 - accuracy: 0.6213
Epoch 8/100
38/38 [==============================] - 148s 4s/step - loss: 0.8647 - accuracy: 0.6413
Epoch 9/100
38/38 [==============================] - 148s 4s/step - loss: 0.9104 - accuracy: 0.6373
Epoch 10/100
38/38 [==============================] - 147s 4s/step - loss: 0.8328 - accuracy: 0.6713

In [ ]:
model.save('./drive/MyDrive/final_project/model/model5.h5') #모델 저장